In [ ]:
pip install arviz

     |████████████████████████████████| 1.6MB 7.6MB/s 
     |████████████████████████████████| 4.7MB 28.8MB/s 
     |████████████████████████████████| 768kB 51.1MB/s 
     |████████████████████████████████| 317kB 43.5MB/s 
  Found existing installation: xarray 0.15.1
    Uninstalling xarray-0.15.1:
      Successfully uninstalled xarray-0.15.1


In [ ]:
import pystan
import numpy as np
import matplotlib.pyplot as plt
import arviz
import json
import pandas as pd

In [ ]:
with open('/content/drive/MyDrive/TFM-Crowd learning/multinomial.json') as json_file:
    data = json.load(json_file)

In [ ]:
data.keys()

dict_keys(['w', 't', 'a', 'k', 't_A', 'w_A', 'ann'])

In [ ]:
print('Un total de '+str(data['w'])+ ' workers')
print('Un total de '+str(data['t'])+ ' tasques')
print('Un total de '+str(data['a'])+' anotacions')
print('Ja que k = '+str(data['k'])+', vol dir que els workers han de triar entre 3 possibles anotacions. Per aquest motiu, la key "ann" només conté 0, 1 o 2.')

Un total de 20 workers
Un total de 1000 tasques
Un total de 10000 anotacions
Ja que k = 3, vol dir que els workers han de triar entre 3 possibles anotacions. Per aquest motiu, la key "ann" només conté 0, 1 o 2.


In [ ]:
dades={'Tasca': data['t_A'], 'Worker': data['w_A'], 'Anotació': data['ann'] }

In [ ]:
df = pd.DataFrame(data=dades)
df

,Tasca,Worker,Anotació
0,0,14,2
1,0,13,2
2,0,4,2
3,0,2,2
4,0,7,1
...,...,...,...
9995,999,17,2
9996,999,2,2
9997,999,13,2
9998,999,14,2


In [ ]:
df.to_csv('crowdsource.csv',index=False)